In [64]:
import re
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn

In [65]:


sentence = 'after waiting what seemed like forever to get into the room . we finally got to see our little one for the first time in 7 weeks and wow what a change it was!'


In [66]:
    
sentence = sentence.lower()
content = re.findall(r"[\w']+|[.,!?;]", sentence)
content


['after',
 'waiting',
 'what',
 'seemed',
 'like',
 'forever',
 'to',
 'get',
 'into',
 'the',
 'room',
 '.',
 'we',
 'finally',
 'got',
 'to',
 'see',
 'our',
 'little',
 'one',
 'for',
 'the',
 'first',
 'time',
 'in',
 '7',
 'weeks',
 'and',
 'wow',
 'what',
 'a',
 'change',
 'it',
 'was',
 '!']

In [67]:


dict_df = pd.read_csv('../TT_data/dictionary.csv')
revdict_df = pd.read_csv('../TT_data/rev_dictionary.csv')

dictionary = dict_df.set_index('0').to_dict()['1']
reverse_dictionary = revdict_df.set_index('0').to_dict()['1']

In [68]:
time_steps = 4

def create_input(sentence):
    #initialize single inputs and total inputs
    instance = []
    location = 0
    
    for location in range(0, len(sentence)):
    
        single_input = [-1]*time_steps
        
        #get the first time_steps words
        words = sentence[location:location+time_steps]

        #if the length of the array is less than time_steps, break
        if len(words) < time_steps:
            break

        #loop the word array
        for index in range(0,time_steps):

            #get the number value for current word
            num = dictionary[words[index]]

            #add to the input array
            single_input[index] = num

        #input the single input array to the total input array
        instance.append(single_input)
        
    return instance

In [69]:
input_instance = np.array(create_input(content))


In [70]:
#sequence length
time_steps=4

#could be any number
num_units=20

#self explanatory
input_size=1

#we need to learn about this
learning_rate=0.001

#57 getsure types + no gesture
n_classes=58
no_gesture_index = 57

#tbd
batch_size=100

In [71]:
#weights and biases of appropriate shape to accomplish above task
out_weights=tf.Variable(tf.random_normal([num_units,n_classes]), name="weights")
out_bias=tf.Variable(tf.random_normal([n_classes]), name="biases")

#defining placeholders
#input image placeholder
x=tf.placeholder("float",[None,time_steps,input_size])
print(x)
#input label placeholder
y=tf.placeholder("float",[None,n_classes])

#processing the input tensor from [batch_size,n_steps,input_size] to "time_steps" number of [batch_size,input_size] tensors
input=tf.unstack(x ,time_steps,1)
print(input)

Tensor("Placeholder_8:0", shape=(?, 4, 1), dtype=float32)
[<tf.Tensor 'unstack_4:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'unstack_4:1' shape=(?, 1) dtype=float32>, <tf.Tensor 'unstack_4:2' shape=(?, 1) dtype=float32>, <tf.Tensor 'unstack_4:3' shape=(?, 1) dtype=float32>]


In [72]:
# defining the network
lstm_layer=rnn.BasicLSTMCell(num_units,forget_bias=1)

with tf.variable_scope("rnn", reuse=None):
    outputs,_=rnn.static_rnn(lstm_layer,input,dtype="float32")
    print(outputs[-1])
    
    #converting last output of dimension [batch_size,num_units] to [batch_size,n_classes] by out_weight multiplication
    prediction=tf.matmul(outputs[-1],out_weights)+out_bias
    print(prediction)

    #loss_function
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
    #optimization
    opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

    #model evaluationmatmul
    correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
    accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

Tensor("rnn_6/rnn/rnn/basic_lstm_cell/mul_11:0", shape=(?, 20), dtype=float32)
Tensor("rnn_6/add:0", shape=(?, 58), dtype=float32)


ValueError: Variable rnn/weights_4/Adam/ does not exist, or was not created with tf.get_variable(). Did you mean to set reuse=None in VarScope?

In [52]:
sess_restore = tf.Session()
saver = tf.train.Saver()
saver.restore(sess_restore, '../datasets/LSTM/model.ckpt')

INFO:tensorflow:Restoring parameters from ../datasets/LSTM/model.ckpt


NotFoundError: Key biases_2 not found in checkpoint
	 [[Node: save_5/RestoreV2_2 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_arg_save_5/Const_0_0, save_5/RestoreV2_2/tensor_names, save_5/RestoreV2_2/shape_and_slices)]]

Caused by op 'save_5/RestoreV2_2', defined at:
  File "/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/src/conda3_runtime/home/envs/DSX-Python35

In [38]:
raw_predictions = prediction.eval(feed_dict={x: input_instance.reshape((-1, time_steps, input_size))}, session=sess_restore).tolist()


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [?,4,1]
	 [[Node: Placeholder = Placeholder[dtype=DT_FLOAT, shape=[?,4,1], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'Placeholder', defined at:
  File "/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/site-p

In [39]:
results = [sublist.index(max(sublist)) for sublist in raw_predictions]

In [40]:
results

[57, 57, 57, 57, 57, 57, 57, 57, 57, 57]